In [18]:
!pwd

/home/b/b382291/git/one_pass/src


In [19]:
import intake

In [112]:
# run mean OPA 
import os
# add directory where the utils.py file is located
import xarray as xr 
import importlib
import dask as dk
import numpy as np 

# for levante
os.chdir('/home/b/b382291/git/one_pass/src')
imported_module = importlib.import_module("meanOPA")

os.chdir('/home/b/b382291/git/one_pass/src')
importlib.reload(imported_module)
from meanOPA import *
from meanOPA import meanOPA # from module (file.py) import class name 

os.chdir('/home/b/b382291/regridder/AQUA')
from aqua import Reader
from aqua.reader import catalogue


# initalising the statistic 
meanStat = meanOPA(meanFreq = "daily", save = "true", saveFreq = "daily")

# want to change to this 
#  stat = "mean", 


In [90]:
# from dask.distributed import LocalCluster, Client, progress
# processes = False
# nworker = 32
# ncpu = 32
# threads = ncpu // nworker
# print(f"Number of CPUs: {ncpu}, number of threads: {threads}, number of workers: {nworker}, processes: {processes}")
# client = Client(processes=processes,
#  threads_per_worker=threads,
#  n_workers=nworker,
#  memory_limit='64GB'
#  )

#catalogue();


os.chdir('/home/b/b382291/regridder/AQUA')

reader = Reader(model="IFS", exp="tco2559-ng5", source="ICMGG_atm2d", regrid="r020")
data = reader.retrieve(fix=False)
tasr = reader.regrid(data["2t"][:,:])

#tasr = reader.regrid(data["2t"][0:16,:])

Testing the output against the np.mean version! 

In [113]:

for i in range(0,30, 5): 
#
    ds1 = tasr.isel(time=slice(i,i+5)) # slice(start, stop, step), extract 'moving window' which is hourly data
    dm = meanStat.mean(ds1)


finished saving
finished saving


In [34]:
dm

<xarray.Dataset>
Dimensions:  (time: 1, lat: 900, lon: 1800)
Coordinates:
  * time     (time) datetime64[ns] 2020-01-21
  * lon      (lon) float64 0.0 0.2 0.4 0.6 0.8 ... 359.0 359.2 359.4 359.6 359.8
  * lat      (lat) float64 -89.9 -89.7 -89.5 -89.3 -89.1 ... 89.3 89.5 89.7 89.9
Data variables:
    meanVar  (time, lat, lon) float64 242.4 242.4 242.4 ... 245.1 245.1 245.1
Attributes: (12/16)
    paramId:                    167
    dataType:                   fc
    numberOfPoints:             26306560
    typeOfLevel:                surface
    stepUnits:                  1
    stepType:                   instant
    ...                         ...
    cfVarName:                  t2m
    missingValue:               9999
    totalNumber:                0
    NV:                         0
    gridDefinitionDescription:  Gaussian Latitude/Longitude Grid
    OPA:                        daily mean calculated using one-pass algorithm

In [68]:
timeStampList = sorted(tasr.time.data) # assuming that incoming data has a time dimension
timeStampPandas = [pd.to_datetime(x) for x in timeStampList]
timeStamp = timeStampPandas[0] # converting to a pandas datetime to calculate if it's the first 
#timeStamp = np.append(timeStamp, timeStamp)

timeStamp[0].strftime("%Y_%m_%d") + "_to_" + timeStamp[-1].strftime("%Y_%m_%d")

#timeStampString = timeStamp[0].strftime("%Y_%m_%d_%H") + "_to_" + timeStamp[-1].strftime("%Y_%m_%d_%H")


TypeError: 'Timestamp' object is not subscriptable

In [80]:
# checking the output against np.mean 

ds2 = tasr.isel(time=slice(0,20)) # slice(start, stop, step), extract 'moving window' which is hourly data

axNum = ds2.get_axis_num('time')
npMean = np.mean(ds2, axis = axNum, dtype = np.float64)
npMean = np.squeeze(npMean)

In [81]:
a = meanStat.meanCum  - npMean
a.values

array([[ 2.84217094e-14,  2.84217094e-14,  2.84217094e-14, ...,
         2.84217094e-14,  5.68434189e-14,  2.84217094e-14],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
         0.00000000e+00,  0.00000000e+00,  5.68434189e-14],
       [-2.84217094e-14,  0.00000000e+00,  0.00000000e+00, ...,
         2.84217094e-14,  0.00000000e+00,  2.84217094e-14],
       ...,
       [ 0.00000000e+00, -2.84217094e-14,  2.84217094e-14, ...,
         0.00000000e+00,  0.00000000e+00,  2.84217094e-14],
       [ 0.00000000e+00,  2.84217094e-14,  0.00000000e+00, ...,
         0.00000000e+00,  0.00000000e+00, -2.84217094e-14],
       [-2.84217094e-14, -2.84217094e-14, -2.84217094e-14, ...,
         2.84217094e-14, -2.84217094e-14,  2.84217094e-14]])

In [114]:
ds = tasr.isel(time=slice(24,25))

In [119]:
import numpy as np
a = meanStat.meanCum - np.squeeze(ds.values)

In [120]:
a.values

array([[ 0.10536476,  0.10536476,  0.10536476, ...,  0.10536476,
         0.10536476,  0.10536476],
       [ 0.2515262 ,  0.2515262 ,  0.2515262 , ...,  0.2515262 ,
         0.2515262 ,  0.2515262 ],
       [ 0.4403712 ,  0.4403712 ,  0.4403712 , ...,  0.4403712 ,
         0.4403712 ,  0.4403712 ],
       ...,
       [ 0.10714496,  0.10714496,  0.10714496, ...,  0.10714496,
         0.10714496,  0.10714496],
       [ 0.0459389 ,  0.0459389 ,  0.0459389 , ...,  0.0459389 ,
         0.0459389 ,  0.0459389 ],
       [-0.05666091, -0.05666091, -0.05666091, ..., -0.05666091,
        -0.05666091, -0.05666091]])